In [1]:
!pip install cassandra-driver


In [2]:

import logger 

# Log the start of the notebook
logger.logging.info('Starting the Cassandra Database operations.')

In [20]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import os


# Ensure these paths are correct
secure_connect_bundle_path = os.path.join(os.getcwd(), 'secure-connect-big-mart.zip')
token_json_path = os.path.join(os.getcwd(), 'your-token.json')

# Load credentials from the token JSON
logger.logging.info('Loading credentials from the token JSON.')
with open('big_mart-token.json') as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

# Configure cluster with secure connect bundle and authentication
cloud_config = {
    'secure_connect_bundle': secure_connect_bundle_path
}
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)

# Create connection to the Cassandra cluster
try:
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    session = cluster.connect()

    # Test query
    row = session.execute("select release_version from system.local").one()
    if row:
        print('-')
        logger.logging.info(f"Cassandra release version: {row[0]}")
    else:
        print('-')
        logger.logging.warning("An error occurred while fetching the release version.")

except Exception as e:
    print('-')
    logger.logging.error(f"An error occurred: {e}")

print(session)

-


In [21]:
# Use the keyspace
logger.logging.info('Setting the keyspace to "sales_prediction".')
session.set_keyspace('sales_prediction')

# Configure paging (adjust fetch size for larger datasets)
session.default_fetch_size = 10000  # Explicitly set the fetch size to handle larger datasets


# Create table for sales data
logger.logging.info('Creating the sales_data table if it doesn\'t exist.')
session.execute("""
    CREATE TABLE IF NOT EXISTS sales_data (
        item_identifier TEXT PRIMARY KEY,
        item_weight FLOAT,
        item_fat_content TEXT,
        item_visibility FLOAT,
        item_type TEXT,
        item_mrp FLOAT,
        outlet_identifier TEXT,
        outlet_establishment_year INT,
        outlet_size TEXT,
        outlet_location_type TEXT,
        outlet_type TEXT,
        item_outlet_sales FLOAT
    )
""")
logger.logging.info("Table created successfully.")


In [6]:
!pip install pandas

In [22]:
import pandas as pd


# Load CSV data from the 'data' folder
logger.logging.info('Loading CSV data from "data/train.csv".')
data = pd.read_csv('../data/train.csv')

# Display the first few rows to verify the data
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [24]:
data.shape

(8523, 12)

In [25]:

# Iterate over the rows in the DataFrame and insert each row into the Cassandra table
logger.logging.info('Inserting rows into the sales_data table.')
for index, row in data.iterrows():
    # Prepare the INSERT query
    query = """
    INSERT INTO sales_data (
        item_identifier, item_weight, item_fat_content, item_visibility, item_type, item_mrp,
        outlet_identifier, outlet_establishment_year, outlet_size, outlet_location_type, outlet_type, item_outlet_sales
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Extract values from the current row and execute the query
    session.execute(query, (
        row['Item_Identifier'], 
        row['Item_Weight'] if pd.notnull(row['Item_Weight']) else None,  # Handle NaN
        row['Item_Fat_Content'], 
        row['Item_Visibility'], 
        row['Item_Type'], 
        row['Item_MRP'], 
        row['Outlet_Identifier'], 
        int(row['Outlet_Establishment_Year']),  # Ensure the year is an integer
        row['Outlet_Size'] if pd.notnull(row['Outlet_Size']) else None,  # Handle NaN
        row['Outlet_Location_Type'], 
        row['Outlet_Type'], 
        row['Item_Outlet_Sales']
    ))

logger.logging.info("Data inserted successfully.")

In [7]:
logger.logging.info("Fetching 10 rows to verify the data.")
rows = session.execute("SELECT * FROM sales_data LIMIT 10")
for row in rows:
    logger.logging.info(f"Row: {row}")
    print(row)

Row(item_identifier='FDX40', item_fat_content='Low Fat', item_mrp=39.91640090942383, item_outlet_sales=38.61640167236328, item_type='Frozen Foods', item_visibility=0.17332421243190765, item_weight=None, outlet_establishment_year=1985, outlet_identifier='OUT019', outlet_location_type='Tier 1', outlet_size='Small', outlet_type='Grocery Store')
Row(item_identifier='FDA58', item_fat_content='Low Fat', item_mrp=235.89320373535156, item_outlet_sales=942.7728271484375, item_type='Snack Foods', item_visibility=0.10375123471021652, item_weight=9.395000457763672, outlet_establishment_year=1997, outlet_identifier='OUT046', outlet_location_type='Tier 1', outlet_size='Small', outlet_type='Supermarket Type1')
Row(item_identifier='FDB05', item_fat_content='Low Fat', item_mrp=247.27760314941406, item_outlet_sales=5944.26220703125, item_type='Frozen Foods', item_visibility=0.08332769572734833, item_weight=5.15500020980835, outlet_establishment_year=1999, outlet_identifier='OUT049', outlet_location_type

In [30]:
rows = session.execute("SELECT * FROM sales_data")
# Convert fetched rows to a list of dictionaries
data_list = [row._asdict() for row in rows]

# Create a DataFrame from the list of dictionaries
import pandas as pd
df = pd.DataFrame(data_list)

logger.logging.info('importing dataset from cassandra database to dataframe')



In [31]:
logger.logging.info(f"Fetched data:\n{df.head()}")
df.head()


,item_identifier,item_fat_content,item_mrp,item_outlet_sales,item_type,item_visibility,item_weight,outlet_establishment_year,outlet_identifier,outlet_location_type,outlet_size,outlet_type
0,FDX40,Low Fat,39.916401,38.616402,Frozen Foods,0.173324,NaN,1985,OUT019,Tier 1,Small,Grocery Store
1,FDA58,Low Fat,235.893204,942.772827,Snack Foods,0.103751,9.395,1997,OUT046,Tier 1,Small,Supermarket Type1
2,FDB05,Low Fat,247.277603,5944.262207,Frozen Foods,0.083328,5.155,1999,OUT049,Tier 1,Medium,Supermarket Type1
3,FDP20,Low Fat,128.102005,1265.020020,Fruits and Vegetables,0.045631,19.850,1987,OUT013,Tier 3,High,Supermarket Type1
4,DRK11,Low Fat,149.939194,1938.809570,Hard Drinks,0.010781,8.210,1999,OUT049,Tier 1,Medium,Supermarket Type1


In [32]:
# Save the DataFrame to a CSV file
csv_file_path = os.path.join(os.getcwd(),'train_df.csv')
df.to_csv(csv_file_path, index=False)
logger.logging.info(f"Data has been saved to {csv_file_path}")

In [33]:

# Close the session and cluster connection
session.shutdown()
cluster.shutdown()
logger.logging.info('shutting down both session and cluster from cassandra database')

In [34]:
df.shape

(1559, 12)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1559 entries, 0 to 1558
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   item_identifier            1559 non-null   object 
 1   item_fat_content           1559 non-null   object 
 2   item_mrp                   1559 non-null   float64
 3   item_outlet_sales          1559 non-null   float64
 4   item_type                  1559 non-null   object 
 5   item_visibility            1559 non-null   float64
 6   item_weight                1273 non-null   float64
 7   outlet_establishment_year  1559 non-null   int64  
 8   outlet_identifier          1559 non-null   object 
 9   outlet_location_type       1559 non-null   object 
 10  outlet_size                1145 non-null   object 
 11  outlet_type                1559 non-null   object 
dtypes: float64(4), int64(1), object(7)
memory usage: 146.3+ KB


In [36]:
df.isna().sum()

item_identifier                0
item_fat_content               0
item_mrp                       0
item_outlet_sales              0
item_type                      0
item_visibility                0
item_weight                  286
outlet_establishment_year      0
outlet_identifier              0
outlet_location_type           0
outlet_size                  414
outlet_type                    0
dtype: int64